In [1]:
!pip install yt_dlp
!pip install streamlit
!pip install torch torchvision torchaudio
!pip install transformers
!pip install streamlit
!pip install ffmpeg-python
!pip install streamlit torch transformers gTTS fpdf diffusers accelerate safetensors
!pip install yt_dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [2]:
%%writefile app.py
import os
import streamlit as st
from transformers import pipeline
import torch
from gtts import gTTS
import tempfile

# Optional: for translation
from transformers import pipeline as hf_pipeline

# --- Streamlit Page Configuration ---
st.set_page_config(page_title="📖 AI Storyteller", layout="centered")

# --- Sidebar Info ---
with st.sidebar:
    st.title("📌 Instructions")
    st.markdown("""
    1. Enter a **story prompt**
    2. Choose **genre**, **tone**, **length**, and **language**
    3. Click **Generate Story**
    4. Enjoy reading or listening to your story! 🎧
    """)
    st.markdown("---")
    st.caption("🌍 Supports multiple languages for narration")

# --- Title ---
st.title("🧠 AI Storyteller with Multilingual Narration")
st.markdown("Let AI craft a story and narrate it in your language!")

# --- Load Story Generator ---
@st.cache_resource
def load_story_model():
    return pipeline(
        "text-generation",
        model="tiiuae/falcon-7b-instruct",
        device=0 if torch.cuda.is_available() else -1
    )

story_gen = load_story_model()

# --- Load Translator Pipeline ---
@st.cache_resource
def load_translators():
    return {
        "hi": hf_pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi"),
        "fr": hf_pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr"),
        "es": hf_pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
    }

translators = load_translators()

# --- Genre & Tone Options ---
genre = st.selectbox("📚 Choose a Genre", ["Fantasy", "Sci-Fi", "Mystery", "Comedy", "Adventure", "Horror"])
tone = st.selectbox("🎭 Select the Tone", ["Light-hearted", "Dramatic", "Suspenseful", "Funny", "Inspiring"])
length = st.slider("🧾 Story Length (in words)", min_value=100, max_value=1000, step=100, value=300)

# --- Language Selection ---
language_options = {
    "English": "en",
    "Hindi": "hi",
    "French": "fr",
    "Spanish": "es"
}
lang_label = st.selectbox("🌐 Choose Narration Language", list(language_options.keys()))
lang_code = language_options[lang_label]

# --- User Prompt ---
prompt = st.text_input("📝 Enter your story prompt:", "A dragon who learns to cook pizza")

# --- Generate Story Function ---
def generate_story(prompt, genre, tone, length):
    instruction = (
        f"story with a {tone.lower()} tone about: {prompt}. "
    )
    result = story_gen(
        instruction,
        max_length=int(length * 1.5),
        do_sample=True,
        top_k=50
    )
    return result[0]["generated_text"]

# --- Translate Story if Needed ---
def translate_story(story_text, target_lang):
    if target_lang == "en":
        return story_text
    elif target_lang in translators:
        translated = translators[target_lang](story_text[:1000])  # limit to 1000 tokens
        return translated[0]['translation_text']
    else:
        return story_text

# --- Text to Speech ---
def text_to_speech(text, lang_code):
    tts = gTTS(text=text, lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmpfile:
        tts.save(tmpfile.name)
        return tmpfile.name

# --- Generate Button ---
if st.button("✨ Generate Story"):
    if not prompt.strip():
        st.warning("Please enter a story prompt.")
    else:
        with st.spinner("Generating your story..."):
            story = generate_story(prompt, genre, tone, length)

        st.subheader("📜 Your AI Story (English)")
        st.write(story)

        with st.spinner(f"Translating story to {lang_label}..."):
            translated_story = translate_story(story, lang_code)

        st.subheader(f"🌍 Translated Story ({lang_label})")
        st.write(translated_story)

        with st.spinner("Generating narration..."):
            audio_path = text_to_speech(translated_story, lang_code)

        st.subheader("🔊 Listen to the Story")
        st.audio(audio_path, format="audio/mp3")

# --- Footer ---
st.markdown("---")
st.caption("© 2025 | Built with ❤️ using Streamlit, Hugging Face, and gTTS")


Writing app.py


In [3]:
# prompt: get my ip address

!wget http://ipinfo.io/ip -qO -

34.126.184.82

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.126.184.82:8501

⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://curvy-cycles-retire.loca.lt
2025-07-05 07:36:45.471351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751701005.494210    2292 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751701005.502533    2292 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-05 07:36:45.525561: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is o